In [ ]:
apt-get install ffmpeg

In [ ]:
!pip install --upgrade git+https://github.com/huggingface/transformers.git accelerate datasets[audio]

In [ ]:
!pip install datasets[audio]

In [ ]:
!pip install transformers[sentencepiece]

In [2]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset

import re

### for serialisation model 

In [4]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
pipe = pipeline(
    "automatic-speech-recognition",
    model='openai/whisper-large-v3',
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    device=device,
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### load and inference

In [6]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

PATH = 'models/whisper'

pipe = pipeline(
    "automatic-speech-recognition",
    model=PATH,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    device=device,
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
result = pipe('data/audio5.mp3')
result['text']
# result['chunks']

Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.


' you you you you Дорогие друзья, рад вас приветствовать на курсе знакомства с языками программирования. Сегодня у нас вводная лекция. Мы поговорим о языках программирования в целом, о языке программирования C-Sharp, об алгоритмах, а также порешаем задачи. Друзья, цель этого курса – познакомить вас с синтаксисом языка программирования высокого уровня, дать представление о подходах к написанию базовых алгоритмов, а также дать представление о подходах к написанию качественного кода. Друзья, перед тем как продолжить и познакомиться с программой настоящего курса, а также с планом работ на сегодня, предлагаю познакомиться и я расскажу немного о себе. Меня зовут Черняев Денис. В компании Geekbrains я работаю уже более года. До прихода в компанию я работал частным преподавателем на порталах дистанционного обучения. Среди преподаваемых предметов я могу выделить языки программирования C, C++, C Sharp, Python. Также могу выделить тематику алгоритмы и структуры данных, операционные системы и комп

### summarize model

In [11]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer
MODEL_NAME = 'cointegrated/rut5-base-absum'
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
model.cuda();
model.eval();

/usr/local/lib/python3.8/dist-packages/huggingface_hub/file_download.py:992: UserWarning: Not enough free disk space to download the file. The expected file size is: 0.00 MB. The target location /root/.cache/huggingface/hub only has 0.00 MB free disk space.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/huggingface_hub/file_download.py:992: UserWarning: Not enough free disk space to download the file. The expected file size is: 0.00 MB. The target location /root/.cache/huggingface/hub/models--cointegrated--rut5-base-absum/blobs only has 0.00 MB free disk space.
  warnings.warn(
config.json: 100%|████████████████████████████████████████████████████████████████████████████████████| 753/753 [00:00<00:00, 446kB/s]
/usr/local/lib/python3.8/dist-packages/huggingface_hub/file_download.py:992: UserWarning: Not enough free disk space to download the file. The expected file size is: 977.27 MB. The target location /root/.cache/huggingface/hub only has 0.00 MB free disk space.
  warnings.

In [12]:
def summarize(
    text, n_words=None, compression=None,
    max_length=1000, num_beams=3, do_sample=False, repetition_penalty=10.0, 
    **kwargs):
    """
    Summarize the text
    The following parameters are mutually exclusive:
    - n_words (int) is an approximate number of words to generate.
    - compression (float) is an approximate length ratio of summary and original text.
    """
    if n_words:
        text = '[{}] '.format(n_words) + text
    elif compression:
        text = '[{0:.1g}] '.format(compression) + text
    x = tokenizer(text, return_tensors='pt', padding=True).to(model.device)
    with torch.inference_mode():
        out = model.generate(
            **x, 
            max_length=max_length, num_beams=num_beams, 
            do_sample=do_sample, repetition_penalty=repetition_penalty, 
            **kwargs)
    return tokenizer.decode(out[0], skip_special_tokens=True)

In [15]:
n = 1000
text = result['text']
res = []

for chunk in [text[i:i+n] for i in range(0, len(text), n)]:
    res.append(summarize(chunk))

print(' '.join(res))

Сегодня у нас вводная лекция. Познакомимся с синтаксисом языка программирования высокого уровня. Сегодня у нас вводная лекция, мы будем говорить о языке C-Sharp. Вспомним основные понятия из курса ведения программирования. Настроим редактор Visual Studio Code. На начальных этапах разработки программ я не рекомендую переходить с разработки на язык со статической типизации. Что вы видите на экране у языков C++, Java? Это отличный выбор для старта программирования. Используйте язык C-Sharp как инструмент создания программ. Вы можете легко перенести на любой другой понравившийся вам язык. Для создания окружения мы будем использовать облачный сервис, который предоставляет вам удаленный рабочий стол. Введите первую часть названия в поиске C-Sharp. Создайте первый проект на языке C Sharp Создайте проект на языке C-sharp. Запустите команду dotnet-new-console, которая создаст файл с исходным кодом programcs. Создадим commit с указанием комментария. Запустим наш код в репозитории Git Вспомним ба

### term finding test

In [19]:
text = result['text']

starts = [m.start() for m in re.finditer('– это', text)]

In [20]:
starts

[5407, 11457, 11524, 11584, 12937, 13833, 13929, 16516, 16925, 38037]

In [21]:
for start in starts:
    print(text[start-30:start+60])
    print()

те на слово, что язык C-Sharp – это отличный выбор для старта программирования, решения ал

руппы терминов. Первая группа – это переменные, типы данных и арифметика. Вторая группа те

етика. Вторая группа терминов – это виды алгоритмических конструкций. Итак, первый термин 

струкций. Итак, первый термин – это переменные. Под переменной мы обычно понимаем некотору

ия. Ну и третий вид алгоритма – это циклический. Циклический алгоритм подразумевает повтор

 основных этапов. Первый этап – это указание типа данных для значения, которое в этой пере

 будет храниться. Второй этап – это указание непосредственно имени переменной или же ее на

з представленных типов данных – это вещественный тип. Он позволяет хранить дробные числа. 

волы. Ну и последний в списке – это логический тип данных. Этот тип данных позволяет сохра

все файлы. Ну и третий пример – это поисковые задачи, в которых требуется найти объект в к



### spacy tests

In [ ]:
!python3.8 -m spacy download ru_core_news_lg

In [ ]:
import spacy
from spacy.lang.ru.examples import sentences 

nlp = spacy.load("ru_core_news_lg")

In [ ]:
text = '''То есть Long позволяет сохранить большее повеличение числа, чем тип данных int. Второй из представленных типов данных – это вещественный тип. Он позволяет хранить дробные числа. Существует две его разновидности. Это типы данных float и double. Отличаются они размером. Тип данных может сохранить большую дробную часть по сравнению, чем тип данных float. Символьный тип данных используется для хранения отдельных символов. Будь то символы латинского алфавита, будь то цифры, будь то специальные символы. Ну и последний в списке – это логический тип данных. Этот тип данных позволяет сохранить всего лишь два значения. Либо логическую истину – true, либо же логически ложное значение – false. Друзья, его мы будем часто использовать в наших условных конструкциях и при создании циклов. Давайте расширим нашу программу и добавим в нее переменные рассмотренных типов данных. Итак, вернемся к проекту Task1 и добавим туда новые переменные. Создадим переменную с вещественным типом данных double, назовем ее, например, d и присвоим ей некоторое значение с дробной частью, например, 5,55.
'''

In [ ]:
doc = nlp(result['text'])
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)